In [ ]:
import boto3
import sys
import pandas as pd
import json
from sqlalchemy import create_engine
import time
S3 = boto3.resource('s3')


def extract_file_as_string(path):
    s3_object = S3.Object('blu-etl', path)
    return s3_object.get()['Body'].read().decode('utf-8')



def connect(credentials_name):
    credentials = json.loads(extract_file_as_string(
        'credentials/' + credentials_name))
    
    return create_engine('postgresql://{0}:{1}@{2}:{3}/{4}'
                        .format(credentials['user'], 
                                credentials['passwd'],
                                credentials['host'],
                                credentials['port'],
                                credentials['dbname']))

In [ ]:
import pandas as pd
import psycopg2
import pandas.io.sql as sqlio
import numpy as np
import os,sys, inspect
from sklearn.preprocessing import LabelEncoder
import db_connections
import boto3
from datetime import datetime
from s3 import *
import utils
import client_score_basic_functions as cs
import payload
import matplotlib.pyplot as plt

from os import walk

import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

con_destiny = db_connections.connect('blu-rds-datascience.json')
con_pipe = db_connections.connect('blu_datascience.json')
con_pagnet_production = db_connections.connect('pagnet_read_replica.json')
con_send=connect('blu-rds-datascience.json')

In [ ]:
base=payload.marchon_historic_base_payload()

In [ ]:
assunto_texto = """<p><span style="color: rgb(0, 0, 0); font-family: Arial; font-size: 13px; font-style: normal; font-variant-ligatures: normal; font-variant-caps: normal; font-weight: normal; letter-spacing: normal; orphans: 2; text-align: left; text-indent: 0px; text-transform: none; white-space: pre-wrap; widows: 2; word-spacing: 0px; -webkit-text-stroke-width: 0px; text-decoration: none; text-decoration-skip-ink: none;">Comprar Marchon, via Blu, &eacute; sua melhor op&ccedil;&atilde;o! Al&eacute;m dos&nbsp;</span><span style="color: rgb(0, 0, 0); font-family: Arial; font-size: 13px; font-style: normal; font-variant-ligatures: normal; font-variant-caps: normal; letter-spacing: normal; orphans: 2; text-align: left; text-indent: 0px; text-transform: none; white-space: pre-wrap; widows: 2; word-spacing: 0px; -webkit-text-stroke-width: 0px; text-decoration: none; text-decoration-skip-ink: none; font-weight: bold;">5%</span><span style="color: rgb(0, 0, 0); font-family: Arial; font-size: 13px; font-style: normal; font-variant-ligatures: normal; font-variant-caps: normal; font-weight: normal; letter-spacing: normal; orphans: 2; text-align: left; text-indent: 0px; text-transform: none; white-space: pre-wrap; widows: 2; word-spacing: 0px; -webkit-text-stroke-width: 0px; text-decoration: none; text-decoration-skip-ink: none;">&nbsp;de desconto que hoje voc&ecirc; j&aacute; tem comprando &agrave; vista, via Blu, agora a Marchon tamb&eacute;m est&aacute; oferecendo&nbsp;</span><span style="color: rgb(0, 0, 0); font-family: Arial; font-size: 13px; font-style: normal; font-variant-ligatures: normal; font-variant-caps: normal; letter-spacing: normal; orphans: 2; text-align: left; text-indent: 0px; text-transform: none; white-space: pre-wrap; widows: 2; word-spacing: 0px; -webkit-text-stroke-width: 0px; text-decoration: none; text-decoration-skip-ink: none; font-weight: bold;">3%</span><span style="color: rgb(0, 0, 0); font-family: Arial; font-size: 13px; font-style: normal; font-variant-ligatures: normal; font-variant-caps: normal; font-weight: normal; letter-spacing: normal; orphans: 2; text-align: left; text-indent: 0px; text-transform: none; white-space: pre-wrap; widows: 2; word-spacing: 0px; -webkit-text-stroke-width: 0px; text-decoration: none; text-decoration-skip-ink: none;">&nbsp;de desconto na modalidade Antecipa&ccedil;&atilde;o Zero! N&atilde;o perca essa oportunidade. Promo&ccedil;&atilde;o por tempo limitado! Antecipe seu saldo futuro sem pagar taxa! Use o saldo das suas vendas parceladas em cart&otilde;es de cr&eacute;dito pela maquininha Blu ou pelo Aplicativo Blu Venda+ sem pagar pela taxa de antecipa&ccedil;&atilde;o! &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;</span> </p>"""
assunto_texto=assunto_texto.replace('\n','')

In [ ]:
base=cs.past_tickets(base, days = 15)

In [ ]:
base=cs.already_talked_about_it(base, 'marchon_na_blu__base_historica')

In [ ]:
base=cs.already_talked_about_it(base, 'marchon_na_blu__já_compram')

In [ ]:
base=cs.today_tickets(base)

In [ ]:
text_as_html=pd.read_csv('text_as_html.csv')

text_as_html['texto'] = assunto_texto  + text_as_html.text

base = base.merge(text_as_html[['client_id','texto']] , on = 'client_id')

In [10]:
def insert_data(data, n = 0):
    
    data['created_at'] =  pd.Timestamp.now()
    data['atribuído'] =  'Rentabilizacao'
    data['ticket_name'] = 'Compre de seus fornecedores e não pague antecipação!'
    data['atribuido_id'] = 360008079852
    data['texto_html'] =   data['texto']
    data['formulario'] =  'rentabilização_fast_add'
    data['formulario_id'] =  360000837612
    data['status'] = 'to_send'
    data['kind'] = 'rentabilização'
    data['subject'] = 'marchon_na_blu__base_historica'
    data['id_zendesk_ticket'] =  np.nan
    data['zendesk_ticket_created_at'] = np.nan
    data['zendesk_user_id'] = np.nan
    data['can_be_send'] = True
    data['subject_kind_id'] = 360030912911
    

    data.head(n).to_sql('zendesk_integration', con_send, index=False, if_exists='append')

In [ ]:
query = """select email email, user_fields_gestor_rentabilizacao from zendeskusers

where 1=1
and user_fields_gestor_rentabilizacao is not  null
--and phone is not null
"""

owner = sqlio.read_sql_query(query, con_pipe)

In [ ]:
base_owner = owner.merge(base, on = 'email')

In [ ]:
df = base_owner.groupby('user_fields_gestor_rentabilizacao', as_index = False).head(30)

In [ ]:
df = df.rename({'user_fields_gestor_rentabilizacao':'cx_owner'}, axis = 1)
df.rename({'text':'texto'}, axis = 1, inplace = True)

In [ ]:
insert_data(df, n = len(df))